In [ ]:
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=False)

In [ ]:
#
# This cell is to be put in python/flashmatch/visualization/view_api.py
# Note import for visualization is different, though, for vis_api.py 
#
import sys
import numpy as np
import plotly.graph_objs as go
from flashmatch import flashmatch, AnalysisManager
from flashmatch.visualization import vis_icarus, icarus_layout3d

class DataManager:
    """
    DataManager class "holds" data retrieved using AnalysisManager. In particular, it holds:
      - "current" entry and event id
      - "cpp" QCluster,Flash_t, etc. in flashmatch.FlashMatchInput format (this is self.cpp attribute)
      - numpy array representation of what's stored in "cpp"
      - Read & update local data attributes is done via self.update function
    TODO: useful properties of DataManager is incremental to AnalysisManager. One can
          either merge two classes or make DataManager inherit from AnalysisManager, if wished.
    """
    def __init__(self):
        self.entry = -1
        self.event = -1
        self.hypothesis_made = False

    def update(self,manager,entry,is_entry=True,make_hypothesis=False):
        """
        Given flashmatch.AnalysisManager, which can interface input data stream and OpT0Finder tools,
        this function "read and update" local data attributes if needed.
        """
        if not is_entry:
            try:
                entry = manager.entry_id(entry)
            except IndexError:
                return False
        if entry < 0 or entry >= len(manager.entries()):
            return False
        if not self.entry == entry:
            self.cpp = manager.make_flashmatch_input(entry)
            self.np_qcluster_v     = [flashmatch.as_ndarray(qcluster) for qcluster in self.cpp.qcluster_v    ]
            self.np_raw_qcluster_v = [flashmatch.as_ndarray(qcluster) for qcluster in self.cpp.raw_qcluster_v]
            self.np_all_pts_v      = [flashmatch.as_ndarray(qcluster) for qcluster in self.cpp.all_pts_v     ]
            self.np_flash_v        = [flashmatch.as_ndarray(flash)    for flash    in self.cpp.flash_v       ]
            pmt_maxid,tpc_maxid=0,0
            for tpc in self.cpp.qcluster_v: tpc_maxid = max(tpc_maxid,tpc.idx)
            for pmt in self.cpp.flash_v: pmt_maxid = max(pmt_maxid,pmt.idx)
            self.np_match_tpc2pmt  = np.ones(shape=(tpc_maxid+1),dtype=np.int32) * -1
            self.np_match_pmt2tpc  = np.ones(shape=(pmt_maxid+1),dtype=np.int32) * -1
            for match in self.cpp.true_match:
                self.np_match_tpc2pmt[match[1]]=match[0]
                self.np_match_pmt2tpc[match[0]]=match[1]
            self.entry = entry
            self.event = manager.event_id(entry)
            self.hypothesis_made = False
        if make_hypothesis and not self.hypothesis_made:
            self.cpp.hypo_v = [manager.flash_hypothesis(qcluster) for qcluster in self.cpp.qcluster_v]
            self.np_hypo_v  = [flashmatch.as_ndarray(flash)       for flash    in self.cpp.hypo_v    ]
            for idx in range(len(self.np_hypo_v)):
                self.cpp.hypo_v[idx].idx = self.cpp.qcluster_v[idx].idx
        return True


class AppManager:
    """
    AppManager serves 2 purposes: 0) it holds data objects which state must be tracked
    through the lifetime of an (stateless) dash html app, and 1) it implements useful
    data parsing methods (e.g. creating a figure, update dropdown options, etc.).
    """
    def __init__(self,cfg,geo,data_particle,data_opflash,dark_mode):
        """
        INPUT:
          - cfg ... OpT0Finder configuration file, passed onto AnalysisManager
          - geo ... vis_icarus (geometry) data file in either PSet or yaml format
          - data_particle ... particle data file stored by ICARUSParticleAna_module
          - data_opflash ... OpFlash data file stored by ICARUSOpFlashAna_module
          - dark_mode ... plotly_dark to be consistent with figure/layout
        """
        self.dark_mode = bool(dark_mode)
        self.ana_manager = AnalysisManager(cfg=cfg, particleana=data_particle, opflashana=data_opflash)
        assert(len(self.ana_manager.entries()))
        self.dat_manager = DataManager()
        self.vis_icarus = vis_icarus(geo)
        self.detector_trace = self.vis_icarus.get_trace_detector()
        self.detector_trace_no_pmt = self.vis_icarus.get_trace_detector(draw_pmts=False)
        self.layout = icarus_layout3d(self.vis_icarus.data(),set_camera=False,dark=self.dark_mode)
        self.empty_view = go.Figure(self.detector_trace,layout=self.layout)
        sys.stdout.flush()


    def qll_score(self,hypothesis,flash):
        res = self.ana_manager.matcher.GetAlgo(flashmatch.kFlashMatch).QLL(hypothesis,flash)
        sys.stdout.flush()
        return np.power(10,-1.*res)
    
    def run_flashmatch(self,qcluster,flash):
        self.ana_manager.matcher.Reset()
        self.ana_manager.matcher.Add(qcluster)
        self.ana_manager.matcher.Add(flash)
        res = self.ana_manager.matcher.Match()
        sys.stdout.flush()
        if not len(res) == 1: return None
        else: return res[0]       
        
    def valid_data_entry(self,entry,is_entry):
        """
        Returns data entry given a valid data index (exist in file) or None
        INPUTS:
          - entry ... either data index ... entry or event id
          - is_entry ... False if the input data_index is event id
        """
        if not is_entry:
            try:
                entry = self.ana_manager.entry_id(entry)
                return entry
            except IndexError:
                return None
        if entry < 0 or entry >= len(self.ana_manager.entries()):
            return None
        return entry

    def current_data_index(self,is_entry):
        """
        Returns current "data index"
        INPUTS:
          - is_entry ... a boolean True=return entry, False=return event id
        OUTPUT:
          - data index integer, either entry or event id
        """
        return self.dat_manager.entry if is_entry else self.dat_manager.event

    def dropdown_qcluster(self,data_index,is_entry):
        """
        Generates a dropdown menu for a dash app to select a QCluster
        INPUTS:
          - data_index ... an integer to specify which entry/event to read
          - is_entry ... True=entry, False=event id
        OUTPUT:
          A list of dicts to be consumed by dash app dropdown options
        """
        if not self.dat_manager.update(self.ana_manager, entry=data_index, is_entry=is_entry):
            return None
        label_v = []
        for idx, qc in enumerate(self.dat_manager.cpp.qcluster_v):
            label = 'Track %02d (%dpts Match=%d T=%.3fus)'
            label = label % (qc.idx, qc.size(), self.dat_manager.np_match_tpc2pmt[qc.idx], qc.time_true)
            label_v.append(label)
        dropdown_qcluster = [dict(label=label_v[idx], value=idx)
                             for idx,qcluster in enumerate(self.dat_manager.np_qcluster_v)]
        dropdown_qcluster += [dict(label='All tracks',value=len(self.dat_manager.np_qcluster_v))]
        return dropdown_qcluster

    def dropdown_flash(self,data_index,is_entry,mode_flash):
        """
        Generates a dropdown menu for a dash app to select a Flash
        INPUTS:
          - data_index ... an integer to specify which entry/event to read
          - is_entry ... True=entry, False=event id
          - mode_flash ... True=Flash, False=Hypothesis
        OUTPUT:
          A list of dicts to be consumed by dash app dropdown options
        """
        if not self.dat_manager.update(self.ana_manager, entry=data_index, is_entry=is_entry, make_hypothesis=(not mode_flash)):
            return None
        target_v = self.dat_manager.np_flash_v if mode_flash else self.dat_manager.np_hypo_v
        label_v = []
        if mode_flash:
            for idx, pmt in enumerate(self.dat_manager.cpp.flash_v):
                pesum = self.dat_manager.np_flash_v[idx].sum()
                label = 'Flash %02d (PE=%de2 Match=%d T=%.3fus MC-T=%.2fus)'
                label = label % (pmt.idx, pesum/100.,
                                 self.dat_manager.np_match_pmt2tpc[pmt.idx], pmt.time, pmt.time_true)
                label_v.append(label)
        else:
            for idx, tpc in enumerate(self.dat_manager.cpp.qcluster_v):
                pesum = self.dat_manager.np_hypo_v[idx].sum()
                label = 'Hypothesis %02d (PE=%de2 MC-T=%.3fus)'
                label = label % (tpc.idx, pesum/100., tpc.time_true)
                label_v.append(label)
        if mode_flash: time_v = [flash.time for flash in self.dat_manager.cpp.flash_v]
        dropdown_flash  = [dict(label=label_v[idx],value=idx)
                            for idx,flash in enumerate(target_v)]
        dropdown_flash += [dict(label='All flashes',value=len(target_v))]
        sys.stdout.flush()
        return dropdown_flash

    def event_display(self, data_index, qcluster_idx_v, flash_idx_v,
                      is_entry, mode_flash, mode_qcluster, use_all_pts, pmt_range):
        """
        Generate 3D display for change in event/entry and/or selection of flash/qcluster
        """
        if not self.dat_manager.update(self.ana_manager, entry=data_index, is_entry=is_entry, make_hypothesis=(not mode_flash)):
            return None
        data = []
        # QCluster
        # if mode_qcluster == 0, then only show qcluster with color-per-cluster
        # if mode_qcluster == 1, then show both qcluster and raw version with 2 colors (red + green)
        # if mode_qcluster == 2, then only show raw qcluster with color-per-cluster

        if qcluster_idx_v is not None and len(qcluster_idx_v):
            if len(self.dat_manager.np_qcluster_v) in qcluster_idx_v:
                qcluster_idx_v = range(len(self.dat_manager.np_qcluster_v))
            idx_v = [self.dat_manager.cpp.qcluster_v[idx].idx for idx in qcluster_idx_v]
            for idx in qcluster_idx_v:
                if mode_qcluster in [0,1]:
                    xyz = self.dat_manager.np_qcluster_v[idx]
                    name = 'Track %02d (%d pts)' % (self.dat_manager.cpp.qcluster_v[idx].idx,len(xyz))
                    trace = go.Scatter3d(x=xyz[:,0],y=xyz[:,1],z=xyz[:,2],mode='markers',
                                         name=name,
                                         #template='plotly' if not self.dark_mode else 'plotly_dark',
                                         marker = dict(size=2, opacity=0.5, color=None if mode_qcluster==0 else 'orange')
                                        )
                    data.append(trace)
                if mode_qcluster in [1,2]:
                    xyz = self.dat_manager.np_raw_qcluster_v[idx] if not use_all_pts else self.dat_manager.np_all_pts_v[idx]
                    name = 'Track (Raw) %02d (%d pts)' % (self.dat_manager.cpp.qcluster_v[idx].idx,len(xyz))
                    trace = go.Scatter3d(x=xyz[:,0],y=xyz[:,1],z=xyz[:,2],mode='markers',
                                         name=name,
                                         #template='plotly' if not self.dark_mode else 'plotly_dark',
                                         marker = dict(size=2, opacity=0.5, color=None if mode_qcluster==2 else 'cyan')
                                        )
                    data.append(trace)
        target_v = self.dat_manager.np_flash_v if mode_flash else self.dat_manager.np_hypo_v
        cpp_target_v = self.dat_manager.cpp.flash_v if mode_flash else self.dat_manager.cpp.hypo_v
        if flash_idx_v and len(flash_idx_v):
            pmt_pos = self.vis_icarus.data()['pmts']
            pmt_val = None
            if len(target_v) in flash_idx_v: pmt_val = np.sum(target_v,axis=0)
            else: pmt_val = np.sum(np.column_stack([target_v[idx] for idx in flash_idx_v]),axis=1)
            pmt_range[0] = np.min(pmt_val) if pmt_range[0].lower() == 'min' else float(pmt_range[0])
            pmt_range[1] = np.max(pmt_val) if pmt_range[1].lower() == 'max' else float(pmt_range[1])
            name = 'Flash (%dE2 PEs)' % int(np.sum(pmt_val)/100.)
            if(len(flash_idx_v)==1 and mode_flash):
                name = 'Flash (%dE2 PEs @ %.2f us)' % (int(np.sum(pmt_val)/100.), int(cpp_target_v[flash_idx_v[0]].time))
            trace = go.Scatter3d(x=pmt_pos[:,0],y=pmt_pos[:,1],z=pmt_pos[:,2],mode='markers',
                                 name=name,
                                 #template='plotly' if not self.dark_mode else 'plotly_dark',
                                 marker = dict(size=6, color=pmt_val, opacity=0.5, cmin=pmt_range[0], cmax=pmt_range[1])
                                )
            data.append(trace)
        elif len(data):
            # no pmt data => show default pmt view
            return go.Figure(self.detector_trace + data, layout=self.layout)
        
        sys.stdout.flush()
        if len(data)<1:
            return self.empty_view
        else:
            return go.Figure(self.detector_trace_no_pmt + data, layout=self.layout)

    def hypothesis_display(self, data_index, qcluster_idx, flash_idx, 
                           pe_range, normalize_pe, is_entry, xpos):
        """
        Generate 3D display of a flash hypothesis for a given QCluster and x-offset
        """
        if not self.dat_manager.update(self.ana_manager, entry=data_index, 
                                       is_entry=is_entry, make_hypothesis=False):
            return None
        data = []
        hypo_val,flash_val,diff_val=None,None,None
        hypo_name,flash_name,diff_name=None,None,None
        if qcluster_idx is not None:
            # Find QCluster
            qcluster = self.dat_manager.cpp.qcluster_v[qcluster_idx]
            raw_qcluster = self.dat_manager.cpp.raw_qcluster_v[qcluster_idx]
            # find x span allowed
            xspan = qcluster.max_x() - qcluster.min_x()
            # if xoffset is larger than allowed range, truncate
            active_bb  = flashmatch.DetectorSpecs.GetME().ActiveVolume()
            max_xpos = min(active_bb.Min()[0] + xspan, active_bb.Max()[0])
            xpos     = max(min(xpos,max_xpos),active_bb.Min()[0])
            # shift to xoffset
            offset_qcluster = flashmatch.QCluster_t(qcluster) - qcluster.min_x() + xpos
            # Make hypothesis
            hypothesis = self.ana_manager.flash_hypothesis(offset_qcluster)
            hypo_val = flashmatch.as_ndarray(hypothesis)
            hypo_name ='Hypothesis (%dE2 PEs)' % int(np.sum(hypo_val)/100.)
            # make a trace for tpc
            xyzv = flashmatch.as_ndarray(offset_qcluster)
            tpc_trace = go.Scatter3d(x=xyzv[:,0],y=xyzv[:,1],z=xyzv[:,2],mode='markers',
                                     name='QCluster (X %.2f)' % xpos,
                                     #template='plotly' if not self.dark_mode else 'plotly_dark',
                                     marker = dict(size=2,opacity=0.5,color='orange')
                                    )
            data.append(tpc_trace)
            # also plot raw all points
            xyzv = self.dat_manager.np_all_pts_v[qcluster_idx]
            tpc_trace = go.Scatter3d(x=xyzv[:,0],y=xyzv[:,1],z=xyzv[:,2],mode='markers',
                                     name='True all points',
                                     #template='plotly' if not self.dark_mode else 'plotly_dark',
                                     marker = dict(size=2,opacity=0.5,color='green')
                                    )
            data.append(tpc_trace)
        
        if flash_idx is not None:
            pmt_pos = self.vis_icarus.data()['pmts']
            flash_val = self.dat_manager.np_flash_v[flash_idx]
            flash_name = 'Flash (%dE2 PEs)' % int(np.sum(flash_val)/100.)

        if hypo_val is not None and flash_val is not None:
            #numerator = (flash_val - hypo_val)
            #denominator = (flash_val + hypo_val)
            #diff_val = np.zeros(shape=numerator.shape,dtype=np.float32)
            #where = np.where(denominator>0)
            #diff_val[where] = numerator[where] / denominator[where] * 2.0
            diff_val = flash_val - hypo_val
            diff_name = 'Diff (Flash-Hypothesis)'

        # Now decide the color range and normalization
        flash_range,hypo_range = (1.e20,-1.e20), (1.e20,-1.e20)
        flash_range = (1.e20,-1.e20) if flash_val is None else (flash_val.min(),flash_val.max())
        hypo_range  = (1.e20,-1.e20) if hypo_val  is None else (hypo_val.min(), hypo_val.max() )
        if normalize_pe == 0:
            if pe_range[0].lower() == 'min':
                pe_range[0] = min(flash_range[0],hypo_range[0])
            else:
                pe_range[0] = float(pe_range[0])
            if pe_range[1].lower() == 'max':
                pe_range[1] = max(flash_range[1],hypo_range[1])
            else:
                pe_range[1] = float(pe_range[1])
        if normalize_pe == 1 and not flash_val is None:
            pe_range = [0.,1.]
            norm = flash_val.sum()
            if not diff_val  is None: diff_val   = diff_val / flash_val
            if not flash_val is None: flash_val /= norm
            if not hypo_val  is None: hypo_val  /= norm
        if normalize_pe == 2 and not hypo_val is None:
            pe_range = [0.,1.]
            norm = hypo_val.sum()
            if not diff_val  is None: diff_val   = diff_val / hypo_val
            if not flash_val is None: flash_val /= norm
            if not hypo_val  is None: hypo_val  /= norm
                
        # make a trace for hypothesis
        pmt_pos = self.vis_icarus.data()['pmts']
        if not hypo_val is None:
            hypo_trace = go.Scatter3d(x=pmt_pos[:,0],y=pmt_pos[:,1],z=pmt_pos[:,2],mode='markers',
                                      name=hypo_name,
                                      #template='plotly' if not self.dark_mode else 'plotly_dark',
                                      marker = dict(size=6,
                                                    color=hypo_val,
                                                    colorscale="OrRd",
                                                    cmin=pe_range[0],cmax=pe_range[1],
                                                    opacity=0.5),
                                      hoverinfo = ['x','y','z','text'],
                                      hovertext = ['%.2f' % val for val in hypo_val]
                                     )
            data.append(hypo_trace)
            
        if not flash_val is None:
            flash_trace = go.Scatter3d(x=pmt_pos[:,0],y=pmt_pos[:,1],z=pmt_pos[:,2],mode='markers',
                                       name=flash_name,
                                       #template='plotly' if not self.dark_mode else 'plotly_dark',
                                       marker = dict(size=6, 
                                                     color=flash_val,
                                                     colorscale = 'Greens',
                                                     cmin=pe_range[0], cmax=pe_range[1],
                                                     opacity=0.5
                                                    ),
                                      hoverinfo = ['x','y','z','text'],
                                      hovertext = ['%.2f' % val for val in flash_val]
                                      )
            data.append(flash_trace)
        
        if not diff_val is None:            
            diff_trace = go.Scatter3d(x=pmt_pos[:,0],y=pmt_pos[:,1],z=pmt_pos[:,2],mode='markers',
                                      name=diff_name,
                                      #template='plotly' if not self.dark_mode else 'plotly_dark',
                                      marker = dict(size=6,
                                                    color=diff_val,
                                                    colorscale=None,
                                                    cmin=pe_range[1]*(-1), cmax=pe_range[1],
                                                    opacity=0.5,),
                                      hoverinfo = ['x','y','z','text'],
                                      hovertext = ['%.2f' % val for val in diff_val]
                                     )
            data.append(diff_trace)
            
        sys.stdout.flush()
        if len(data)<1:
            return self.empty_view
        return go.Figure(self.detector_trace_no_pmt + data, layout=self.layout)
    
    def run_qll(self, data_index, qcluster_idx, flash_idx, is_entry, xpos):
        """
        Generate 3D display of a flash hypothesis for a given QCluster and x-offset
        """
        if not self.dat_manager.update(self.ana_manager, entry=data_index, 
                                       is_entry=is_entry, make_hypothesis=False):
            return None
        data = []
        if qcluster_idx is None or flash_idx is None:
            return None
        # Find QCluster
        qcluster = self.dat_manager.cpp.qcluster_v[qcluster_idx]
        raw_qcluster = self.dat_manager.cpp.raw_qcluster_v[qcluster_idx]
        # find x span allowed
        xspan = qcluster.max_x() - qcluster.min_x()
        # if xoffset is larger than allowed range, truncate
        active_bb  = flashmatch.DetectorSpecs.GetME().ActiveVolume()
        max_xpos = min(active_bb.Min()[0] + xspan, active_bb.Max()[0])
        xpos     = max(min(xpos,max_xpos),active_bb.Min()[0])
        # shift to xoffset
        offset_qcluster = flashmatch.QCluster_t(qcluster) - qcluster.min_x() + xpos
        # Make hypothesis
        hypothesis = self.ana_manager.flash_hypothesis(offset_qcluster)
        # Find flash
        flash = self.dat_manager.cpp.flash_v[flash_idx]

        # Run QLL
        score_xoffset = self.qll_score(hypothesis,flash)
        match = self.run_flashmatch(qcluster,flash)
        sys.stdout.flush()
        return raw_qcluster.min_x(), score_xoffset, match


In [ ]:
#
# This cell is to be put in python/flashmatch/visualization/view_data.py
#
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc

def view_data(cfg,geo,data_particle,data_opflash,dark_mode=True,port=5000):
    """
    Visualized OpT0Finder input data as well as FlashHypothesis for varying x-position
    INPUT:
      - cfg ... OpT0Finder configuration file, passed onto AnalysisManager
      - geo ... vis_icarus (geometry) data file in either PSet or yaml format
      - data_particle ... particle data file stored by ICARUSParticleAna_module
      - data_opflash ... OpFlash data file stored by ICARUSOpFlashAna_module
      - dark_mode ... use dark theme dash app
      - port ... set the custom port id for dash app
    """
    _manager = AppManager(cfg=cfg, geo=geo, data_particle=data_particle, data_opflash=data_opflash, dark_mode=dark_mode)
    entry = 0
    xmin = _manager.vis_icarus.data()['tpc0'][0][0]
    xmax = _manager.vis_icarus.data()['tpc1'][1][0]

    #
    # Create app
    #
    #app = dash.Dash('Flash Match Event Viewer')#,external_stylesheets=[dbc.themes.DARKLY])
    app = dash.Dash('Flash Match Event Viewer',
                    external_stylesheets= ['https://cdnjs.cloudflare.com/ajax/libs/animate.css/3.5.2/animate.min.css)']
                   )
    header_text_style = dict(fontFamily='Georgia', fontWeight=300, fontSize=26)
    span_text_style = dict(fontFamily='Georgia', fontWeight=600, fontSize=24, color='navy')
    label_text_style = dict(fontFamily='Georgia', fontWeight=300, fontSize=20)
    para_text_style = dict(fontFamily='Georgia', fontWeight=300, fontSize=16)
    app.layout = html.Div([
        html.H2('Flash Matching Data Viewer', style=header_text_style),

        html.Div([html.Label('View data entry: (0-%d)' % (len(_manager.ana_manager.entries())-1),
                             style=label_text_style)
                 ],style={'padding': '5px'}),

        html.Div([dcc.RadioItems(id='entry_or_event',
                                 options=[dict(label='Entry Number',value='entry'),
                                          dict(label='Event ID',value='event')],
                                 value='entry'),
                  dcc.Input(id='data_index', value=str(entry), type='int')],
                 style={'width': '100%', 'display': 'inline-block', 'padding': '5px'}),
        html.Div(id='error_entry'),
        #
        # TPC options
        #
        html.Div([html.Span('Select QCluster_t to display', style=span_text_style),
                 ],style={'padding': '5px'}),
        html.Div([dcc.Dropdown(id='select_qcluster',
                               options=_manager.dropdown_qcluster(entry,is_entry=True),
                               multi=True),
                 ],style={'padding': '5px'}),
        html.Div([html.Label('Show also raw QClusters?',
                             style=label_text_style),
                  dcc.RadioItems(id='mode_qcluster',
                                 options=[dict(label='No',value='no_raw_qcluster'),
                                          dict(label='With Raw QCluster',value='with_raw_qcluster'),
                                          dict(label='Only Raw QCluster',value='only_raw_qcluster')],
                                 value='no_raw_qcluster'),],
                 style={'width': '50%', 'display': 'inline-block', 'padding': '5px'}),
        html.Div([html.Label('Raw QCluster to be shown should be:',
                             style=label_text_style),
                  dcc.RadioItems(id='use_all_pts',
                                 options=[dict(label='Before X-shift',value='raw_qcluster'),
                                          dict(label='... and before active BB cut',value='all_pts')],
                                 value='raw_qcluster'),],
                 style={'width': '50%', 'display': 'inline-block', 'padding': '5px'}),

        #
        # PMT options
        #
        html.Div([html.Span('Select Flash_t to display', style=span_text_style),
                 ],style={'padding': '5px'}),
        html.Div([dcc.Dropdown(id='select_flash',
                               options=_manager.dropdown_flash(entry,is_entry=True,mode_flash=True),
                               multi=True),
                 ],style={'padding': '5px'}),
        html.Div([html.Label('View PMTs with OpFlash or Hypothesis?',
                             style=label_text_style),
                  dcc.RadioItems(id='mode_flash',
                                 options=[dict(label='Flash',value='flash'),
                                          dict(label='Hypothesis',value='hypothesis')],
                                 value='flash'),],
                 style={'width': '50%', 'display': 'inline-block', 'padding': '5px'}),
        html.Div([html.Label('PMT color scale: ', style=label_text_style),
                  html.Br(),
                  dcc.Input(id='pmt_cmin', value='min', type='str'),# style={'padding':'5px'}),
                  html.Label(' to ', style=label_text_style),
                  dcc.Input(id='pmt_cmax', value='max', type='str'),# style={'padding':'5px'}),
                  html.Label(' (type "min" and "max" for automatic range setting)', style=para_text_style),
                 ], style={'width': '100%', 'display': 'inline-block', 'padding': '5px'}),
        html.Div(id='error_pmt_range'),
        dcc.Loading(id="loading_static_display", children=[html.Div(id="wait_static_display")], type="default"),
        html.Div([dcc.Graph(id='visdata',figure=_manager.empty_view)],
                  style={'padding':'5px'}),
        #
        # Hypothesis event display
        #
        html.H2('Hypothesis playground: move QCluster along x!',style=header_text_style),

        html.Div([html.Span('Select QCluster to display', style=span_text_style),
                 ],style={'padding': '5px'}),
        html.Div([dcc.Dropdown(id='target_qcluster',
                               options=_manager.dropdown_qcluster(entry,is_entry=True),
                               multi=False),
                 ],style={'padding': '5px'}),
        html.Div([html.Span('Select Flash_t to match', style=span_text_style),
                 ],style={'padding': '5px'}),
        html.Div([dcc.Dropdown(id='target_flash',
                               options=_manager.dropdown_flash(entry,is_entry=True,mode_flash=True),
                               multi=False),
                 ],style={'padding': '5px'}),
        html.Div([html.Label('Normalize PE scale for score calculation',
                             style=label_text_style),
                  dcc.RadioItems(id='normalize_pe',
                                 options=[dict(label='No',value='0'),
                                          dict(label='Normalize to hypothesis',value='1'),
                                          dict(label='Normalize to flash',value='2')],
                                 value='0'),],
                 style={'width': '50%', 'display': 'inline-block', 'padding': '5px'}),
        html.Div([html.Label('PMT color scale: ', style=label_text_style),
                  html.Br(),
                  dcc.Input(id='flash_cmin', value='min', type='str'),# style={'padding':'5px'}),
                  html.Label(' to ', style=label_text_style),
                  dcc.Input(id='flash_cmax', value='max', type='str'),# style={'padding':'5px'}),
                  html.Label(' (type "min" and "max" for automatic range setting)', style=para_text_style),
                 ], style={'width': '100%', 'display': 'inline-block', 'padding': '5px'}),
        html.Div(id='error_flash_range'),
        html.Div([html.Label('X position ', style=label_text_style),
                  #dcc.Input(id='xoffset_text', value='min', type='str')
                 ],style={'padding': '5px'}),
        html.Div([dcc.Slider(id='xoffset',min=xmin,max=xmax,step=.1,value=xmin),
                 ],style={'padding': '5px'}),
        #dcc.Loading(id="loading-playdata",
        #            children=[html.Div([dcc.Graph(id='playdata',figure=_manager.empty_view)],
        #                               style={'padding': '5px'})],
        #            type="circle"),
        html.Div(id="match_result"),
        dcc.Loading(id="loading_dynamic_display", children=[html.Div(id="wait_dynamic_display")], type="default"),
        html.Div([dcc.Graph(id='playdata',figure=_manager.empty_view)],style={'padding': '5px'}),
    ],style={'width': '80%', 'display': 'inline-block', 'vertical-align': 'middle'})

    #
    # call backs
    #
    @app.callback(dash.dependencies.Output("error_entry", "children"),
                  [dash.dependencies.Input("entry_or_event","value"),
                   dash.dependencies.Input("data_index","value")])
    def error_entry(entry_or_event,data_index):
        is_entry = entry_or_event == 'entry'
        if data_index is None:
            raise dash.exceptions.PreventUpdate
        if not data_index.isdigit():
            return [html.Label("Invalid %s ID %s (not an integer)" % (entry_or_event,data_index),
                               style={'color':'red'})]
        data_index = int(data_index)
        if _manager.valid_data_entry(data_index,is_entry) is None:
            return [html.Label("Invalid %s ID %d (data does not exist)" % (entry_or_event,data_index),
                               style={'color':'red'})]
        else:
            return ""

    @app.callback(dash.dependencies.Output("error_pmt_range", "children"),
                  [dash.dependencies.Input("pmt_cmin","value"),
                   dash.dependencies.Input("pmt_cmax","value")])
    def error_entry(pmt_cmin,pmt_cmax):
        pmt_cmin = str(pmt_cmin).lower()
        pmt_cmax = str(pmt_cmax).lower()
        cmin_good,cmax_good=True,True
        try:
            if not pmt_cmin == "min":
                pmt_cmin = float(pmt_cmin)
        except ValueError:
            return [html.Label('Invalid PMT min value: %s (either numerical value or "min")' % pmt_cmin,
                                style={'color':'red'})]
        try:
            if not pmt_cmax == "max":
                pmt_cmax = float(pmt_cmax)
        except ValueError:
            return [html.Label('Invalid PMT max value: %s (either numerical value or "min")' % pmt_cmax,
                                style={'color':'red'})]
        return ""
    
    @app.callback(dash.dependencies.Output("error_flash_range", "children"),
              [dash.dependencies.Input("flash_cmin","value"),
               dash.dependencies.Input("flash_cmax","value")])
    def error_entry(flash_cmin,flash_cmax):
        flash_cmin = str(flash_cmin).lower()
        flash_cmax = str(flash_cmax).lower()
        cmin_good,cmax_good=True,True
        try:
            if not flash_cmin == "min":
                flash_cmin = float(flash_cmin)
        except ValueError:
            return [html.Label('Invalid PMT min value: %s (either numerical value or "min")' % flash_cmin,
                                style={'color':'red'})]
        try:
            if not flash_cmax == "max":
                flash_cmax = float(flash_cmax)
        except ValueError:
            return [html.Label('Invalid PMT max value: %s (either numerical value or "min")' % flash_cmax,
                                style={'color':'red'})]
        return ""


    @app.callback(dash.dependencies.Output("wait_static_display", "children"),
                  [dash.dependencies.Input("mode_flash", "value")])
    def wait_static_display(mode):
        if mode == 'hypothesis':
            from flashmatch import phot
            print('hypothesis mode chosen, loading photon library...')
            phot.PhotonVisibilityService.GetME().LoadLibrary()
        return ""
    
    @app.callback(dash.dependencies.Output("wait_dynamic_display", "children"),
                  [dash.dependencies.Input("target_qcluster", "value")])
    def wait_dynamic_display(target_qcluster):
        if target_qcluster is not None:
            from flashmatch import phot
            print('hypothesis mode chosen, loading photon library...')
            phot.PhotonVisibilityService.GetME().LoadLibrary()
        return ""

    @app.callback(dash.dependencies.Output("select_qcluster","options"),
                  [dash.dependencies.Input("entry_or_event","value"),
                   dash.dependencies.Input("data_index","value")])
    def update_dropdown_select_qcluster(entry_or_event,data_index):
        """
        Update drop-down "select_qcluster" options for QCluster when event/entry is changed
        """
        is_entry = entry_or_event == 'entry'
        if data_index is None:
            raise dash.exceptions.PreventUpdate
        data_index=int(data_index)
        if _manager.valid_data_entry(data_index,is_entry) is None:
            raise dash.exceptions.PreventUpdate
        options = _manager.dropdown_qcluster(data_index=int(data_index),is_entry=is_entry)
        if options is None:
            print('Cannot find data for',entry_or_event,data_index)
            raise dash.exceptions.PreventUpdate
        else: return options


    @app.callback(dash.dependencies.Output("target_qcluster","options"),
                  [dash.dependencies.Input("entry_or_event","value"),
                   dash.dependencies.Input("data_index","value")])
    def update_dropdown_target_qcluster(entry_or_event,data_index):
        """
        Update drop-down "target_qcluster" options for QCluster when event/entry is changed
        """
        is_entry = entry_or_event == 'entry'
        if data_index is None:
            raise dash.exceptions.PreventUpdate
        data_index=int(data_index)
        if _manager.valid_data_entry(data_index,is_entry) is None:
            raise dash.exceptions.PreventUpdate
        options = _manager.dropdown_qcluster(data_index=int(data_index),is_entry=is_entry)
        if options is None: raise dash.exceptions.PreventUpdate
        else: return options[:-1]


    @app.callback(dash.dependencies.Output("select_flash","options"),
                  [dash.dependencies.Input("entry_or_event","value"),
                   dash.dependencies.Input("data_index","value"),
                   dash.dependencies.Input("mode_flash","value")])
    def update_dropdown_select_flash(entry_or_event,data_index,mode_flash):
        """
        Update drop-down "select_flash" options for Flash when event/entry is changed
        """
        is_entry = entry_or_event == 'entry'
        mode_flash = mode_flash == 'flash'
        if data_index is None:
            raise dash.exceptions.PreventUpdate
        data_index=int(data_index)
        if _manager.valid_data_entry(data_index,is_entry) is None:
            raise dash.exceptions.PreventUpdate
        options = _manager.dropdown_flash(data_index=int(data_index),
                                          is_entry=is_entry,
                                          mode_flash=mode_flash)
        if options is None: raise dash.exceptions.PreventUpdate
        else: return options

    @app.callback(dash.dependencies.Output("target_flash","options"),
                  [dash.dependencies.Input("entry_or_event","value"),
                   dash.dependencies.Input("data_index","value")])
    def update_dropdown_target_flash(entry_or_event,data_index):
        """
        Update drop-down "target_flash" options for Flash when event/entry is changed
        """
        is_entry = entry_or_event == 'entry'
        if data_index is None:
            raise dash.exceptions.PreventUpdate
        data_index=int(data_index)
        if _manager.valid_data_entry(data_index,is_entry) is None:
            raise dash.exceptions.PreventUpdate
        options = _manager.dropdown_flash(data_index=int(data_index),
                                          is_entry=is_entry,
                                          mode_flash=True)
        if options is None: raise dash.exceptions.PreventUpdate
        else: return options[:-1]

    @app.callback(dash.dependencies.Output("visdata","figure"),
                  [dash.dependencies.Input("select_qcluster","value"),
                   dash.dependencies.Input("select_flash","value"),
                   dash.dependencies.Input("mode_flash","value"),
                   dash.dependencies.Input("mode_qcluster","value"),
                   dash.dependencies.Input("use_all_pts","value"),
                   dash.dependencies.Input("pmt_cmin","value"),
                   dash.dependencies.Input("pmt_cmax","value"),
                   dash.dependencies.Input("entry_or_event","value"),
                   dash.dependencies.Input("data_index","value")]
                 )
    def update_static(select_qcluster, select_flash, mode_flash, mode_qcluster, use_all_pts,
                      pmt_cmin, pmt_cmax,
                      entry_or_event, data_index):
        """
        Update "visdata" 3D display for change in event/entry and/or selection of flash/qcluster
        """
        is_entry = entry_or_event == 'entry'
        mode_flash = mode_flash == 'flash'
        if mode_qcluster == 'no_raw_qcluster': mode_qcluster = 0
        elif mode_qcluster == 'with_raw_qcluster': mode_qcluster=1
        elif mode_qcluster == 'only_raw_qcluster': mode_qcluster=2
        else: raise ValueError
        use_all_pts = True if use_all_pts == 'all_pts' else False
        if data_index is None: raise dash.exceptions.PreventUpdate
        data_index=int(data_index)
        if _manager.valid_data_entry(data_index,is_entry) is None:
            raise dash.exceptions.PreventUpdate
        pmt_range=[0,0]
        if pmt_cmin.lower()=='min':
            pmt_range[0]='min'
        else:
            try:
                pmt_range[0]=str(float(pmt_cmin))
            except ValueError:
                raise dash.exceptions.PreventUpdate
        if pmt_cmax.lower()=='max': pmt_range[1]='max'
        else:
            try:
                pmt_range[1]=str(float(pmt_cmax))
            except ValueError:
                raise dash.exceptions.PreventUpdate
        fig = _manager.event_display(int(data_index), select_qcluster, select_flash,
                                     is_entry, mode_flash, mode_qcluster, use_all_pts, pmt_range)
        if fig is None: raise dash.exceptions.PreventUpdate
        else: return fig


    @app.callback(dash.dependencies.Output("playdata","figure"),
                  [dash.dependencies.Input("target_qcluster","value"),
                   dash.dependencies.Input("target_flash","value"),
                   dash.dependencies.Input("xoffset","value"),
                   dash.dependencies.Input("flash_cmin","value"),
                   dash.dependencies.Input("flash_cmax","value"),
                   dash.dependencies.Input("normalize_pe","value"),
                   dash.dependencies.Input("entry_or_event","value"),
                   dash.dependencies.Input("data_index","value")]
                 )
    def update_dynamic(target_qcluster, target_flash, xoffset, 
                       flash_cmin, flash_cmax, normalize_pe,
                       entry_or_event, data_index):
        """
        Update "playdata" 3D display for change in event/entry and/or selection of flash/qcluster
        """
        is_entry = entry_or_event == 'entry'
        if data_index is None or (target_qcluster is None and target_flash is None):
            raise dash.exceptions.PreventUpdate
        data_index=int(data_index)
        if _manager.valid_data_entry(data_index,is_entry) is None:
            raise dash.exceptions.PreventUpdate
        flash_range=[0,0]
        if flash_cmin.lower()=='min': flash_range[0]='min'
        else:
            try:
                flash_range[0]=str(float(flash_cmin))
            except ValueError:
                raise dash.exceptions.PreventUpdate
        if flash_cmax.lower()=='max': flash_range[1]='max'
        else:
            try:
                flash_range[1]=str(float(flash_cmax))
            except ValueError:
                raise dash.exceptions.PreventUpdate
        fig = _manager.hypothesis_display(int(data_index), target_qcluster, target_flash,
                                          flash_range, int(normalize_pe),
                                          is_entry, float(xoffset))
        if fig is None: raise dash.exceptions.PreventUpdate
        else: return fig
        
    @app.callback(dash.dependencies.Output("match_result","children"),
                  [dash.dependencies.Input("target_qcluster","value"),
                   dash.dependencies.Input("target_flash","value"),
                   dash.dependencies.Input("xoffset","value"),
                   dash.dependencies.Input("entry_or_event","value"),
                   dash.dependencies.Input("data_index","value"),]
                 )
    def update_match(target_qcluster, target_flash, xoffset, entry_or_event, data_index):
        """
        Update "playdata" 3D display for change in event/entry and/or selection of flash/qcluster
        """
        is_entry = entry_or_event == 'entry'
        if data_index is None or target_qcluster is None or target_flash is None:
            raise dash.exceptions.PreventUpdate
        data_index=int(data_index)
        if _manager.valid_data_entry(data_index,is_entry) is None:
            raise dash.exceptions.PreventUpdate
        qll = _manager.run_qll(int(data_index), target_qcluster, target_flash,
                               is_entry, float(xoffset))
        show_style_text = dict(fontFamily='Georgia', fontWeight=300, fontSize=20)
        show_style_num  = dict(fontFamily='Georgia', fontWeight=600, fontSize=20, color='blue')
        
        xoffset = float(xoffset)
        res = []
        if qll is None:
            raise dash.exceptions.PreventUpdate
        else:
            # xmin and score_xoffset should not be None
            xmin,score_xoffset,match = qll
            res += [html.Label('QLL joint probability ',style=show_style_text),
                    html.Label('%.4f' % score_xoffset,style=show_style_num),
                    html.Label(' at X ',style=show_style_text),
                    html.Label('%.2f' % xoffset,style=show_style_num),
                    html.Label(' ... true X ',style=show_style_text),
                    html.Label('%.2f' % xmin,style=show_style_num),
                    html.Br()]
            if match is not None:
                res += [html.Label('QLL match result: score ',style=show_style_text),
                        html.Label('%.4f' % match.score, style=show_style_num),
                        html.Label(' at X ',style=show_style_text),
                        html.Label('%.2f' % match.tpc_point.x,style=show_style_num),
                        html.Label(' ... %.1fms %dsteps (scanned %.2f to %.2f)' % (match.duration/1.e6,
                                                                                   match.num_steps,
                                                                                   match.minimizer_min_x,
                                                                                   match.minimizer_max_x))]

        return res

    app.server.run(port=port)

In [ ]:
cfg = '../dat/flashmatch.cfg'
geo = '../dat/detector_specs.cfg'
data_particle = '../particleana_000.root'
data_opflash = '../opflashana_000.root'
view_data(cfg,geo,data_particle,data_opflash,port=5004)